## 7.1 언어 모델을 사용한 문장 생성

### 7.1.1 RNN을 사용한 문장 생성의 순서

![image](https://github.com/choibigo/Study/assets/38881179/097c329a-0b7c-416e-af2f-65e2a387a526)

- 시계열 데이터 T개분을 모아 처리하는 TimeLSTM 과 Affine 계층으로 구성
- 만약 입력으로 "I"가 주어지면 1개의 LSTM 계층에서 다음에 출현하는 단어의 확률분포를 출력한다.

![image](https://github.com/choibigo/Study/assets/38881179/ea536885-b3e0-4f3a-8c56-06570c0f9f45)

- 단어를 새로 생성하는 방법을 살펴본다.
- 첫 번째, 확률이 가장 높은 던어를 선택하는 방법을 떠올릴 수 있다, 확률이 가장 높은 단어를 선택할 뿐이므로 결과가 일정하게 정해지는 '결정적'인 방법이다.
- 또한 '확률적'으로 선택하는 방법도 있다, 각 후보 단어의 확률(Softmax 통과 확률)에 맞게 선택하는 것으로, 확률이 높은 단어가 선택되기 쉽고, 확률이 낮은 단어는 선택되기 어렵다.
- 이 방식은 선택되는 단어(샘플링 단어)가 매번 다를 수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/b6a4f306-f859-44e5-8e9e-71ef7d813fc6)

- 확률 분포로 부터 샘플링을 수행하여 'say'가 선택될 수 있다.
- 'say'의 확률이 가장 높아서가 아닌 확률 분포에서 샘플링한 결과인것을 주의 하자

![image](https://github.com/choibigo/Study/assets/38881179/5ad5e962-50b9-4e4c-834b-d9bb15d82a08)

- 샘플링을 반복하여 ```<eos>```와 같이 종결 기호가 나올때 까지 반복한다.
- 그러면 새로운 문장을 생성할 수 있다.
- 여기서 주목할 것은 이렇게 생성한 문장은 훈련데이터에는 존재하지 않는 문장이다, 따라서 말 그대로 새롭게 생성된 문장이다.
- 모델이 새로 훈련 데이터에서 사용된 단어의 ```정렬 패턴```을 학습한 것이므로 새로 생성한 문장도 자연스럽고 의미가 통하는 문장일 것이라 기대한다.

#### 7.1.2 문장 생성 구현

In [4]:
import sys
sys.path.append('..')
from common.time_layers import TimeEmbedding, TimeLSTM, TimeAffine, TimeSoftmaxWithLoss
import pickle
import numpy as np
from numpy.random import randn as rn
from data_set import ptb
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.utill import eval_perplexity

class Rnnlm:
    def __init__(self, vocab_size=10000, wordvec_size=100, hidden_size=100):
        V, D, H = vocab_size, wordvec_size, hidden_size

        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype(np.float64) # 전체 단어 수 에서 압축 할 차원수로 가기 위한 Weight
        lstm_Wx = (rn(D, 4*H)/np.sqrt(D)).astype(np.float64) # Embedding된 Vector 차원 수 => hidden state 4개(f, i, o, g)로 되기 위한 Weight
        lstm_Wh = (rn(H, 4*H)/np.sqrt(H)).astype(np.float64) # Hidden State State => 4개의 Hidden state 가 되기 위한 Weight 
        lstm_b = np.zeros(4 * H).astype(np.float64) # W들 연산 이후 더해질 Bias
        affine_W = (rn(H, V)) # LSTM 각 layer에서 나온 Output의 Affine 연산을 한번에 계산하기 위한 TimeAffine Layer의 Weight Hidden state 수 => 분류할 총 vocab 크기
        affine_b = np.zeros(V).astype(np.float64)

        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layer = self.layers[1]

        # 모든 가중치와 기울기를 리스트에 모은다.
        self.params = []
        self.grads = []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs):
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs
    
    def forward(self, xs, ts):
        score = self.predict(xs)
        loss = self.loss_layer.forward(score, ts)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        self.lstm_layer.reset_state()
    
    def save_params(self, file_name='Rnnlm.pkl'):
        with open(file_name, 'wb') as f:
            pickle.dumps(self.params, f)
    
    def load_params(self, file_name='Rnnlm.pkl'):
        with open(file_name, 'rb') as f:
            self.params = pickle.load(f)

In [19]:
import sys
sys.path.append('..')
from common.time_layers import *
import numpy as np
from common.base_model import BaseModel
from numpy.random import randn as rn


class BetterRnnlm(BaseModel):
    def __init__(self, vocab_size=10000, wordvec_size=650, hidden_size=650, dropout_ratio=0.5):
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        embed_W = (rn(V, D) / 100).astype(np.float64)
        lstm_Wx1 = (rn(D, 4*H) / np.sqrt(D)).astype(np.float64)
        lstm_Wh1 = (rn(H, 4*H) / np.sqrt(H)).astype(np.float64)
        lstm_b1 = np.zeros(4*H).astype(np.float64)
        lstm_Wx2 = (rn(D, 4*H) / np.sqrt(D)).astype(np.float64)
        lstm_Wh2 = (rn(H, 4*H) / np.sqrt(H)).astype(np.float64)
        lstm_b2 = np.zeros(4*H).astype(np.float64)
        affine_b = np.zeros(V).astype(np.float64)

        self.layers = [
            TimeEmbedding(embed_W),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx1, lstm_Wh1, lstm_b1, stateful=True),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx2, lstm_Wh2, lstm_b2, stateful=True),
            TimeDropout(dropout_ratio),
            TimeAffine(embed_W.T, affine_b)
        ] 
        # dropout을 각 layer 마다 추가함
        # Affine과 Embedding Weight를 공유함
        # LSTM계층을 깊게 쌓음

        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layers = [self.layers[2], self.layers[4]]
        self.drop_layers = [self.layers[1], self.layers[3], self.layers[5]]
        
        self.params = []
        self.grads = []

        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs, train_flg=False):
        for layer in self.drop_layers:
            layer.train_flg = train_flg
        for layer in self.layers:
            xs = layer.forward(xs)

        return xs

    def forward(self, xs, ts, train_flg=True):
        score = self.predict(xs, train_flg)
        loss = self.loss_layer.forward(score, ts)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        for layer in self.lstm_layers:
            layer.reset_state()

In [20]:
import sys
sys.path.append('..')
import numpy as np
from common.function import softmax

class RnnlmGen(BetterRnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        # start_id : 최초로 주어지는 단어 ID
        # sample_size : 샘플링 하는 단어수 (만드는 문장의 단어수)
        # skip_ids : 단어 ID 리스트로서 이 리스트에 속하는 단어는 샘플링 되지 않도록 한다, <unk>나 N 등 전처리된 단어를 샘플링 하지 않게 하는 용도로 사용한다.

        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1) # predict 구현이 배치로 구현되어 있기 때문에 1개만 처리 하더라도 1x1로 형태를 변경한다.
            score = self.predict(x) # 예측된 단어의 Score를 호출 한다.
            p = softmax(score.flatten()) # 출력된 Score를 softmax를 통해서 확률로 변환한다.

            sampled = np.random.choice(len(p), size=1, p=p) # 확률분포 p로 부터 데이터를 샘플링한다.
            if (skip_ids is None) or (sampled not in skip_ids): # skip_ids에 sampling된 id가 있다면 건너 뛴다.
                x = sampled # 출력을 입력으로 변경한다.
                word_ids.append(int(x))

        return word_ids


In [25]:
import sys
sys.path.append('..')
from data_set import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
# model.load_params('./Rnnlm.pkl')
model.load_params('./BetterRnnlm.pkl')

start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# Generate Sentence
word_ids = model.generate(start_id, skip_ids, sample_size=10)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you think but paper difficult into that.
 with women
